# Introduction

The purpose of this task is to extract and save information about hockey teams into JSON format, based on data from files located in the `/data/raw` directory, which were generated in the previous stage. The information to be scraped and saved includes:  

- Team Name (`Team Name`),  
- Year (`Year`),  
- Number of wins (`Wins`),  
- Number of losses (`Losses`),  
- Number of overtime losses (`OT Losses` - Overtime Losses),  
- Win percentage (`Win %`),  
- Number of goals scored (`Goals For (GF)`),  
- Number of goals conceded (`Goals Against (GA)`),  
- Goal differential (`+ / -`).  

Each collected record should be organized into a dictionary with the structure shown below and then added to the results list:  

```python  
{  
    'Team Name': 'Boston Bruins',  
    'Year': '1990',  
    'Wins': '44',  
    'Losses': '24',  
    'OT Losses': '',  
    'Win %': '0.55',  
    'Goals For (GF)': '299',  
    'Goals Against (GA)': '264',  
    '+ / -': '35'  
}  
```

Place each item into the results list.

The resulting data should be saved in a file named hockey_teams.json, which will be placed in the `data/interim/` folder. This file will serve as a data source for further analysis in the next part of the workshop.

> At this point, converting HTML to JSON may seem complex and unnecessary, but it aims to consolidate knowledge regarding this data structure due to its universality and prevalence not only in the world of data analysis but generally in IT as well.


# Notebook Configuration

## Import Required Libraries

In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob
import json

In [3]:
from glob import glob
raw_files = sorted(glob("data/raw/hockey_teams_page_*.html"))

# Scraping

To scrape the required information from the saved files, follow these steps:

1. Find all HTML files in the `data/raw` folder using the `glob` module.
2. For each HTML file, use `BeautifulSoup` to scrape the page and extract the needed data.
3. Save the obtained data as partially processed in the `hockey_teams.json` file located in the `/data/interim/` folder.

These steps will allow for efficient processing of data from HTML files and prepare them for further analysis.

## List of HTML files

Using the `glob` module, find all `html` files in the `data/raw` folder.

In [6]:
RAW_DIR = Path("data/raw")
INTERIM_DIR = Path("data/interim")
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

html_files = sorted(RAW_DIR.glob("*.html"))

print(f"Nalezeno HTML souborů: {len(html_files)}")
for f in html_files[:25]:
    print(" -", f.name)

Nalezeno HTML souborů: 25
 - hockey_teams_page_01.html
 - hockey_teams_page_02.html
 - hockey_teams_page_03.html
 - hockey_teams_page_04.html
 - hockey_teams_page_05.html
 - hockey_teams_page_06.html
 - hockey_teams_page_07.html
 - hockey_teams_page_08.html
 - hockey_teams_page_09.html
 - hockey_teams_page_10.html
 - hockey_teams_page_11.html
 - hockey_teams_page_12.html
 - hockey_teams_page_13.html
 - hockey_teams_page_14.html
 - hockey_teams_page_15.html
 - hockey_teams_page_16.html
 - hockey_teams_page_17.html
 - hockey_teams_page_18.html
 - hockey_teams_page_19.html
 - hockey_teams_page_20.html
 - hockey_teams_page_21.html
 - hockey_teams_page_22.html
 - hockey_teams_page_23.html
 - hockey_teams_page_24.html
 - hockey_teams_page_25.html


## Scraping

Extract data from `html` files, making sure to maintain the expected structure of a single record:

```python
{
    'Team Name': 'Boston Bruins',
    'Year': '1990',
    'Wins': '44',
    'Losses': '24',
    'OT Losses': '',
    'Win %': '0.55',
    'Goals For (GF)': '299',
    'Goals Against (GA)': '264',
    '+ / -': '35'
}
```

In [8]:
def parse_html_file(path: Path):
    """Vrátí list záznamů z jedné HTML stránky ve formátu zadaných klíčů."""
    html = path.read_text(encoding="utf-8", errors="ignore")
    soup = BeautifulSoup(html, "html.parser")

    table = soup.find("table")
    if table is None:
        return []

    headers = [th.get_text(strip=True) for th in table.select("thead th")]
    if not headers:
        first_tr = table.find("tr")
        headers = [cell.get_text(strip=True) for cell in first_tr.find_all(["th", "td"])]

    records = []
    for tr in table.select("tbody tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]

        if not any(cells):
            continue

        if len(cells) < len(headers):
            cells += [""] * (len(headers) - len(cells))
        elif len(cells) > len(headers):
            cells = cells[:len(headers)]

        record = dict(zip(headers, cells))
        records.append(record)

    return records


all_records = []
for fp in html_files:
    all_records.extend(parse_html_file(fp))

print(f"Počet záznamů: {len(all_records)}")

for r in all_records[:2]:
    print(r)

Počet záznamů: 607
{'Team Name': 'Boston Bruins', 'Year': '1990', 'Wins': '44', 'Losses': '24', 'OT Losses': '', 'Win %': '0.55', 'Goals For (GF)': '299', 'Goals Against (GA)': '264', '+ / -': '35'}
{'Team Name': 'Buffalo Sabres', 'Year': '1990', 'Wins': '31', 'Losses': '30', 'OT Losses': '', 'Win %': '0.388', 'Goals For (GF)': '292', 'Goals Against (GA)': '278', '+ / -': '14'}


# Summary

After extracting the relevant information, the final step in preparation for analysis is to save the data to disk.

### Saving the file
Here, save the data to `data/interim/` and name the file `hockey_teams.json`

> Note: Remember to import the appropriate library for handling the JSON format beforehand.

In [9]:
INTERIM_DIR = Path("data/interim")
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

out_path = INTERIM_DIR / "hockey_teams.json"

import json
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(all_records, f, ensure_ascii=False, indent=2)

print(f"Uloženo: {out_path} | Počet záznamů: {len(all_records)}")

Uloženo: data\interim\hockey_teams.json | Počet záznamů: 607


In [10]:
import pandas as pd

df = pd.DataFrame(all_records)

csv_path = INTERIM_DIR / "hockey_teams.csv"

df.to_csv(csv_path, index=False, encoding="utf-8")

print(f"CSV uloženo: {csv_path}")
print(f"Počet řádků: {len(df)}")
print(f"Sloupce: {list(df.columns)}")

df.head()

CSV uloženo: data\interim\hockey_teams.csv
Počet řádků: 607
Sloupce: ['Team Name', 'Year', 'Wins', 'Losses', 'OT Losses', 'Win %', 'Goals For (GF)', 'Goals Against (GA)', '+ / -']


,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,,0.55,299,264,35
1,Buffalo Sabres,1990,31,30,,0.388,292,278,14
2,Calgary Flames,1990,46,26,,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,,0.425,273,298,-25


ZV – Zpětná vazba / Review
Co je dobře
Je vidět postupné zpřesňování kódu: první verze bez ošetření, pak varianta s posunem results.append(...) dovnitř smyčky, nakonec verze s try/except a kontrolou chybějící buňky. Takhle se to má učit.
Struktura extrakce je konzistentní. Všude se držíš pojmenovaných tříd z HTML (name, year, wins, losses, pct, gf, ga, diff), takže je kód čitelný i pro někoho, kdo HTML nezná.
U poslední verze máš správně ošetřené pole ot-losses pomocí podmínky if ot_losses else "". To je přesně ten typ chyby, který u reálného scraperu dělá největší problémy.
Ukládáš do JSONu s indent=2 a ensure_ascii=False, takže výstup je jednak čitelný, jednak zvládá diakritiku.
Na konci děláš kontrolu načtení přes pandas.read_json(...). To je dobrá praxe: nenechat se ukolébat tím, že se soubor vytvořil, ale ověřit, že jde znovu načíst.
Co zlepšit
Máš v jednom souboru tři až čtyři téměř stejné bloky kódu (načti HTML → BeautifulSoup → find_all("tr", class_="team") → smyčka → ulož JSON). To by v produkčním kódu mělo být jen jednou. Navrhuj funkční strukturu:
def load_html(path): ...
def parse_teams(html): ...
def save_json(data, path): ...
Pak v notebooku jen skládáš kroky, místo aby ses posouvala copy-paste stylem.
Cesty jsou natvrdo na Windows:
r"C:\Users\rdpre\Workshop_-_files\..."
To je v pořádku pro tvůj počítač, ale pro sdílený repozitář nebo hodnocení je lepší použít Path a relativní cestu k repozitáři:
from pathlib import Path
base = Path("data")
html_path = base / "raw" / "hockey_teams_page_001.html"
json_path = base / "interim" / "hockey_teams.json"
Umožní to spouštět notebook i ostatním.
V první verzi kódu máš chybu v odsazení: results.append({...}) je až za smyčkou, takže by se přidával jen poslední tým. Později jsi to opravila. Ve zpětné vazbě to zdůrazni studentům: u scrapingu jsou chyby v odsazení fatální, protože se vytvoří „správný“ JSON, ale jen s jedním záznamem.
V několika blocích tiskneš celé results (print(results) nebo dokonce print(results[:582])). U reálnějších dat to rychle znepřehlední výstup notebooku. Vhodnější je:
print(f"Uloženo {len(results)} záznamů")
print(results[:3])
Když už máš try/except AttributeError, je dobré si chybné řádky logovat i s obsahem řádku (ne jen pořadí). Usnadní to debugging:
except AttributeError:
    print(f"Chyba při zpracování řádku {i+1}: {team}")
V JSONu střídáš názvy klíčů: jednou Goals for, pak Goals For (GF), jednou Goals against, pak Goals Against (GA). Pro následnou analýzu v Pandasu je lepší držet jeden styl, ideálně snake_case:
"team_name": ...,
"year": ...,
"wins": ...,
"losses": ...,
"ot_losses": ...,
"win_pct": ...,
"goals_for": ...,
"goals_against": ...,
"goal_diff": ...
nebo alespoň konzistentně stejná diakritika a velká písmena.